<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Generate Readme for Awesome Notebooks

In [1]:
import os
import naas_drivers
import urllib.parse

## Variables

In [2]:
readme_template = "README_template.md"
readme = "README.md"
replace_var = "[[DYNAMIC_LIST]]"
current_file = '.'
notebook_ext = '.ipynb'
github_url = 'https://github.com/jupyter-naas/awesome-notebooks/tree/master'

## Get files list

In [3]:
total = []
for root, directories, files in os.walk(current_file, topdown=False):
    total.append({"root": root, "directories":directories, "files":files})
total.sort(key=lambda x: x.get('root'))

## Convert filepath in Markdown text

In [4]:
def get_file_md(folder_nice, folder_url, files, title_sep="##", subtitle_sep="*"):
    md = ""
    if (len(files) > 0):
        md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
    #                 print(file)

            if file.endswith(notebook_ext):
                file_url = urllib.parse.quote(file)
                file_nice = file.replace('_', ' ')
                file_nice = file_nice.replace(notebook_ext, '')
                file_nice = file_nice.replace(folder_nice, '')
                file_nice = file_nice.strip()
                file_nice = file_nice[0].capitalize() + file_nice[1:]
                md += f"{subtitle_sep} [{file_nice}]({github_url}/{folder_url}/{file_url})\n"
    return md

## Create list of all notebooks

In [5]:
generated_list = ""

for cur in total:
    root = cur.get('root')
    md_round = ""
    directories = cur.get('directories') 
    files = cur.get('files')
    if ('.git' not in root and '.ipynb_checkpoints' not in root and '.' != root):
#         print(root)
#         print(root, files, '\n')
        folder_nice = root.replace('./', '')
        folder_url = urllib.parse.quote(folder_nice)
        if ('/' not in folder_nice):
            md_round += get_file_md(folder_nice, folder_url, files)
        elif ('/' in folder_nice):
            folder_url = urllib.parse.quote(folder_nice)
            subfolder_nice = folder_nice.split('/')[1].replace('_', ' ').replace(folder_nice, '').strip()
            md_round += get_file_md(subfolder_nice, folder_url, files, "\t###", "\t-")
    elif ('.ipynb_checkpoints' in root):
#         print(root, files)
        for file in files:
            try:
                os.remove(os.path.join(root, file))
            except:
                pass
        try:
            os.rmdir(root)
        except:
            pass
#     print(md_round)
    generated_list += md_round
      

## Preview the generated list

In [6]:
naas_drivers.markdown.display(generated_list)

In [7]:
template = open(readme_template).read()

template = template.replace(replace_var, generated_list)

f  = open(readme, "w+")
f.write(template)
f.close()